In [1]:
import tensorflow as tf

2023-06-05 01:16:52.626677: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
import pandas as pd

In [3]:
smiles_with_rdkit = pd.read_csv("..//Data//combined_data_with_rdkit.csv")

In [4]:
smiles_with_rdkit.shape

(2272938, 211)

In [5]:
smiles_with_rdkit = smiles_with_rdkit.dropna()

In [6]:
smiles_with_rdkit.shape

(2263096, 211)

In [7]:
smiles_with_rdkit = smiles_with_rdkit.drop_duplicates(["ChEMBL ID"])

In [8]:
smiles_with_rdkit.shape

(2260219, 211)

In [9]:
# smiles_with_rdkit["ChEMBL ID"].nunique()

In [10]:
# read in the small molecule phase 3 data
smaller_data = pd.read_csv("..//Data//Small_Molecule_Phase_3.csv", on_bad_lines="skip", sep = ";")

In [11]:
smaller_data.head()

,ChEMBL ID,Name,Synonyms,Type,Max Phase,Molecular Weight,Targets,Bioactivities,AlogP,Polar Surface Area,...,Heavy Atoms,HBA (Lipinski),HBD (Lipinski),#RO5 Violations (Lipinski),Molecular Weight (Monoisotopic),Np Likeness Score,Molecular Species,Molecular Formula,Smiles,Inchi Key
0,CHEMBL4297324,REMIMAZOLAM TOSYLATE,REMIMAZOLAM TOSILATE|REMIMAZOLAM TOSYLATE,Small molecule,3.0,611.52,NaN,NaN,4.18,69.37,...,28,6,0,0,438.0691,-0.59,NEUTRAL,C28H27BrN4O5S,COC(=O)CC[C@@H]1N=C(c2ccccn2)c2cc(Br)ccc2-n2c(...,UNLWPYSYFQLJSV-LMOVPXPDSA-N
1,CHEMBL4297424,RUBIDIUM RB-82,RUBIDIUM (82 RB)|RUBIDIUM (RB-82)|RUBIDIUM RB-...,Small molecule,3.0,81.92,NaN,NaN,None,None,...,None,None,None,None,84.9118,None,None,Rb,[82Rb],IGLNJRXAVVLDKE-OIOBTWANSA-N
2,CHEMBL545437,AMIBEGRON HYDROCHLORIDE,AMIBEGRON HCL|Amibegron hydrochloride|SR-58611...,Small molecule,3.0,440.37,4.0,10.0,3.46,67.79,...,28,5,2,0,403.1550,-0.80,BASE,C22H27Cl2NO4,CCOC(=O)COc1ccc2c(c1)C[C@@H](NC[C@H](O)c1cccc(...,NQIZCDQCNYCVAS-RQBPZYBGSA-N
3,CHEMBL3039533,GRAZOPREVIR,GRAZOPREVIR,Small molecule,3.0,784.93,NaN,NaN,3.30,195.22,...,54,15,3,2,766.3360,0.23,ACID,C38H52N6O10S,C=C[C@@H]1C[C@]1(NC(=O)[C@@H]1C[C@@H]2CN1C(=O)...,RXSARIJMSJWJLZ-CIAYNJNFSA-N
4,CHEMBL4297407,ALTROPANE I-123,(123I)ALTROPANE|123I-ALTROPANE|ALTROPANE I-123...,Small molecule,3.0,425.27,NaN,NaN,3.88,29.54,...,23,3,0,0,429.0601,0.19,NEUTRAL,C18H21FINO2,COC(=O)[C@H]1[C@@H](c2ccc(F)cc2)C[C@@H]2CC[C@H...,GTQLIPQFXVKRKJ-HYRAKNMCSA-N


In [12]:
smaller_data.shape

(1110, 33)

In [13]:
smaller_data = smaller_data.drop_duplicates(["ChEMBL ID"])

In [14]:
smaller_data.shape

(1110, 33)

In [15]:
# remove smaller data samples from smiles with rdkit
smiles_with_rdkit_wo_small_phase_3 = smiles_with_rdkit[~smiles_with_rdkit["ChEMBL ID"].isin(smaller_data["ChEMBL ID"].values)]

In [16]:
smiles_with_rdkit_wo_small_phase_3.shape

(2259260, 211)

In [17]:
features = smiles_with_rdkit_wo_small_phase_3.drop(["ChEMBL ID", "AlogP", "Polar Surface Area"], axis = 1)

In [18]:
features.head()

,Smiles,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,O=C(OCc1ccco1)C1=C(c2ccc(Cl)c(Cl)c2)CCC1,12.295307,12.295307,0.142845,-0.296322,0.713880,337.202,323.090,336.032000,114.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CCOC(=O)c1csc2nc(-c3ccccc3)c(-c3ccc(S(C)(=O)=O...,12.491187,12.491187,0.235375,-3.305778,0.444525,426.519,408.375,426.070799,148.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Cc1cccnc1CN(C)C[C@H]1Cc2c(cccc2N2CCNCC2)CN1,4.547914,4.547914,0.482022,0.482022,0.847856,365.525,334.277,365.257946,144.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,N#CCC1(n2cc(-c3ncnc4[nH]ccc34)cn2)CN(C2CCN(Cc3...,14.045580,14.045580,0.355489,-0.548433,0.445375,488.546,462.338,488.224849,184.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,COC(CNC(=O)c1cc(C(=O)c2c(-c3c(Cl)cccc3Cl)noc2C...,13.161794,13.161794,0.146870,-0.576299,0.397212,452.294,433.142,451.070176,158.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
targets = smiles_with_rdkit_wo_small_phase_3[["AlogP", "Polar Surface Area"]]

In [20]:
targets.head()

,AlogP,Polar Surface Area
0,5.27,39.44
1,4.31,77.74
2,1.95,43.43
3,3.69,89.66
4,3.86,106.45


In [21]:
import matplotlib.pyplot as plt

In [22]:
# plt.hist(targets)
# plt.show()

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
x_train, x_valid, y_train, y_valid = train_test_split(features, targets, test_size = 0.25, shuffle = True, 
                                                     random_state = 42)

In [25]:
x_train.head()

,Smiles,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
653009,CN(C(=O)COC(=O)c1ccc(C(C)(C)C)cc1)C1CCS(=O)(=O)C1,12.116300,12.116300,0.011179,-3.066142,0.758054,367.467,342.267,367.145344,138.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2201368,CCOP(=O)(OCC)C(CC1CCCC(C)C1=O)P(=O)(OCC)OCC,13.483605,13.483605,0.049496,-3.788657,0.405067,412.400,378.128,412.177977,154.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
217562,CC1CN(c2ccc(Nc3ncc4c(n3)n(C3CCCC3)c(=O)c3nnnn3...,13.203823,13.203823,0.099583,-0.210295,0.472687,460.546,432.322,460.244756,176.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1289860,CN1C(=O)[C@@H](O)[C@@H](c2ccccc2)[C@@H]1[C@@H]...,12.199822,12.199822,0.352931,-1.128924,0.906376,297.354,278.202,297.136493,114.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1489876,CCC(=O)NCC(C)(C)C(=O)N1CCCC[C@H]1C(=O)O[C@H](C...,13.608971,13.608971,0.105869,-0.840324,0.388304,538.685,496.349,538.304287,212.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
x_train.isnull().sum().sum()

0

In [27]:
x_valid.isnull().sum().sum()

0

In [28]:
from sklearn.feature_selection import VarianceThreshold

In [29]:
vt = VarianceThreshold(1)

In [30]:
x_train_rdkit_selected = vt.fit_transform(x_train.iloc[:,1:])

In [31]:
from sklearn.preprocessing import StandardScaler

In [32]:
std = StandardScaler()

In [33]:
x_train_rdkit_selected = std.fit_transform(x_train_rdkit_selected)

In [34]:
import pickle

In [35]:
pickle.dump(std, open('..//Data//std_scaler.pkl', 'wb'))

In [36]:
x_valid_rdkit_selected = vt.transform(x_valid.iloc[:,1:])

In [37]:
x_valid_rdkit_selected = std.transform(x_valid_rdkit_selected)

In [38]:
x_train_rdkit_selected.shape

(1694445, 104)

In [39]:
x_train.shape

(1694445, 208)

In [40]:
# vt.variances_

In [41]:
# len(vt.get_feature_names_out(vt.feature_names_in_))

In [42]:
x_train = pd.concat([x_train[["Smiles"]].reset_index(drop = True), pd.DataFrame(x_train_rdkit_selected)], axis = 1)

In [43]:
x_train.shape

(1694445, 105)

In [44]:
x_valid.shape

(564815, 208)

In [45]:
x_valid = pd.concat([x_valid[["Smiles"]].reset_index(drop = True), pd.DataFrame(x_valid_rdkit_selected)], 1)

/tmp/ipykernel_1763297/1857118243.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x_valid = pd.concat([x_valid[["Smiles"]].reset_index(drop = True), pd.DataFrame(x_valid_rdkit_selected)], 1)


In [46]:
x_valid.shape

(564815, 105)

In [47]:
# x_train.isnull().sum().sum()

In [48]:
## save
x_train.to_csv("..//Data//x_train.csv", index = False)
x_valid.to_csv("..//Data//x_valid.csv", index = False)

In [49]:
y_train.to_csv("..//Data//y_train.csv", index = False)
y_valid.to_csv("..//Data//y_valid.csv", index = False)

In [50]:
# remove smaller data samples from smiles with rdkit
smiles_with_rdkit_with_small_phase_3 = smiles_with_rdkit[smiles_with_rdkit["ChEMBL ID"].isin(smaller_data["ChEMBL ID"].values)]

In [51]:
# smiles_with_rdkit_with_small_phase_3.shape

In [52]:
smiles_with_rdkit_with_small_phase_3_features, smiles_with_rdkit_with_small_phase_3_outputs = \
smiles_with_rdkit_with_small_phase_3.drop(["ChEMBL ID", "AlogP", "Polar Surface Area"], axis = 1), \
smiles_with_rdkit_with_small_phase_3[["ChEMBL ID", "AlogP", "Polar Surface Area"]]                     

In [53]:
smiles_with_rdkit_with_small_phase_3_features.shape

(959, 208)

In [54]:
smiles_with_rdkit_with_small_phase_3_features_selected = vt.transform(smiles_with_rdkit_with_small_phase_3_features.iloc[:,1:])

In [55]:
smiles_with_rdkit_with_small_phase_3_features_selected = std.transform(smiles_with_rdkit_with_small_phase_3_features_selected)

In [56]:
smiles_with_rdkit_with_small_phase_3_features = pd.concat([smiles_with_rdkit_with_small_phase_3_features[["Smiles"]].reset_index(drop = True), pd.DataFrame(smiles_with_rdkit_with_small_phase_3_features_selected)], 1)

/tmp/ipykernel_1763297/3565475699.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  smiles_with_rdkit_with_small_phase_3_features = pd.concat([smiles_with_rdkit_with_small_phase_3_features[["Smiles"]].reset_index(drop = True), pd.DataFrame(smiles_with_rdkit_with_small_phase_3_features_selected)], 1)


In [57]:
smiles_with_rdkit_with_small_phase_3_features.shape

(959, 105)

In [58]:
smiles_with_rdkit_with_small_phase_3_outputs.shape

(959, 3)

In [59]:
# len(smiles_with_rdkit_with_small_phase_3_features.columns == features.columns)

In [60]:
smiles_with_rdkit_with_small_phase_3_features.to_csv("..//Data//smiles_with_rdkit_with_small_phase_3_features.csv", index = False)
smiles_with_rdkit_with_small_phase_3_outputs.to_csv("..//Data//smiles_with_rdkit_with_small_phase_3_outputs.csv", index = False)

In [61]:
# smiles_with_rdkit_with_small_phase_3.to_csv("smiles_with_rdkit_with_small_phase_3.csv", index = False)